In [5]:
!pip3 install --upgrade pip3
!pip3 install pandas
!pip3 install fuzzywuzzy
!pip3 install sentence-transformers
!pip3 install torch
!pip3 install virtualenv
!pip3 install ipykernel
!pip3 install notebook

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement pip3 (from versions: none)
ERROR: No matching distribution found for pip3
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation

In [11]:
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [41]:
import pandas as pd
from fuzzywuzzy import fuzz
from sentence_transformers import SentenceTransformer, util
from collections import defaultdict
import json
import numpy as np

# Option 1: Disable truncation for all columns
pd.set_option('display.max_colwidth', None)

# Option 2: Set maximum column width to a very large number
pd.set_option('display.max_colwidth', 1000)


In [ ]:
# pip33 install virtualenv
# python3 -m venv st 
# source st/bin/activate

# alias python=python3.10  # as env has multiple python versions 3.10 and 3.13
# alias pip3=pip33.10

# pip3 install torch
# pip3 install sentence_transformers

## Functions

In [35]:
def safe_str_conversion(value):
    """
    Safely convert value to string, handling None and complex types
    """
    if value is None:
        return ''
    try:
        return str(value)
    except:
        return repr(value)

def semantic_similarity(text1, text2, threshold=0.7):
    """
    Calculate semantic similarity between two texts
    """
    # Convert to strings, handling None values
    text1 = safe_str_conversion(text1)
    text2 = safe_str_conversion(text2)
    
    # If both texts are empty, return 0
    if not text1.strip() and not text2.strip():
        return 0
    
    try:
        embeddings = model.encode([text1, text2], convert_to_tensor=True)
        cosine_sim = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
        return cosine_sim
    except Exception as e:
        print(f"Semantic similarity calculation error: {e}")
        return 0

def calculate_metrics(human_value, llm_value, threshold=0.7):
    """
    Calculate metrics for comparing human and LLM values
    Handles different types and null values
    """
    # Early exit for None values
    if human_value is None and llm_value is None:
        return {
            "match": "N/A",
            "similarity": "0%",
            "precision": 0,
            "recall": 0,
            "is_populated": False
        }
    
    # Handle cases with one None value
    if human_value is None or llm_value is None:
        return {
            "match": "✗",
            "similarity": "0%",
            "precision": 0,
            "recall": 0,
            "is_populated": False
        }
    
    # Numeric comparison
    if isinstance(human_value, (int, float)) and isinstance(llm_value, (int, float)):
        match = np.isclose(human_value, llm_value, rtol=1e-5)
        similarity = 1.0 if match else 0.0
        return {
            "match": "✓" if match else "✗",
            "similarity": f"{similarity * 100}%",
            "precision": similarity,
            "recall": similarity,
            "is_populated": True
        }
    
    # List comparison
    if isinstance(human_value, list) and isinstance(llm_value, list):
        # Check for list of dictionaries
        if all(isinstance(x, dict) for x in human_value) and \
           all(isinstance(x, dict) for x in llm_value):
            return calculate_dict_list_metrics(human_value, llm_value, threshold)
        
        # Regular list comparison
        return calculate_list_metrics(human_value, llm_value, threshold)
    
    # String or other type comparison
    similarity = semantic_similarity(human_value, llm_value, threshold)
    match = similarity >= threshold
    
    return {
        "match": "✓" if match else "✗",
        "similarity": f"{similarity * 100:.2f}%",
        "precision": similarity,
        "recall": similarity,
        "is_populated": True
    }

def calculate_list_metrics(human_value, llm_value, threshold=0.7):
    """
    Calculate metrics for lists with semantic similarity
    """
    human_value = [safe_str_conversion(x) for x in human_value]
    llm_value = [safe_str_conversion(x) for x in llm_value]
    
    true_positives = 0
    false_negatives = len(human_value)
    false_positives = len(llm_value)
    
    for human_item in human_value:
        if not human_item.strip():
            false_negatives -= 1
            continue
        
        best_match = max(
            (semantic_similarity(human_item, llm_item) for llm_item in llm_value if llm_item.strip()),
            default=0
        )
        
        if best_match >= threshold:
            true_positives += 1
            false_negatives -= 1
            false_positives -= 1

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    
    return {
        "match": "✓" if precision == 1 and recall == 1 else "✗",
        "similarity": f"{(precision + recall) * 50:.2f}%",
        "precision": precision,
        "recall": recall,
        "is_populated": True
    }

def calculate_dict_list_metrics(human_value, llm_value, threshold=0.7):
    """
    Calculate metrics for lists of dictionaries
    """
    true_positives = 0
    false_negatives = len(human_value)
    false_positives = len(llm_value)
    
    for human_dict in human_value:
        found_match = False
        for llm_dict in llm_value:
            # Ensure all human_dict keys exist in llm_dict
            if all(human_key in llm_dict for human_key in human_dict):
                matches = all(
                    semantic_similarity(str(human_dict[key]), str(llm_dict.get(key, ''))) >= threshold
                    for key in human_dict
                )
                
                if matches:
                    true_positives += 1
                    false_negatives -= 1
                    false_positives -= 1
                    found_match = True
                    break
        
        if not found_match:
            continue
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    
    return {
        "match": "✓" if precision == 1 and recall == 1 else "✗",
        "similarity": f"{(precision + recall) * 50:.2f}%",
        "precision": precision,
        "recall": recall,
        "is_populated": True
    }

def compare_flat_dictionaries(human_data, llm_data, threshold=0.7):
    """
    Compare two flat dictionaries with different value types
    """
    metrics_data = []
    overall_precision = 0
    overall_recall = 0
    populated_llm_count = 0
    populated_human_count = 0

    # Ensure inputs are dictionaries
    if not isinstance(human_data, dict) or not isinstance(llm_data, dict):
        raise ValueError("Both human_data and llm_data must be dictionaries")

    for key in human_data.keys():
        human_value = human_data.get(key)
        llm_value = llm_data.get(key)
        
        try:
            # Calculate metrics
            metrics = calculate_metrics(human_value, llm_value, threshold)
            
            # Collect metrics data
            metrics_data.append({
                "Key": key,
                "Human Value": safe_str_conversion(human_value),
                "LLM Value": safe_str_conversion(llm_value),
                "Match": metrics['match'],
                "Similarity": metrics['similarity'],
                "Precision": metrics['precision'],
                "Recall": metrics['recall']
            })
            
            # Update counts and metrics
            if llm_value is not None:
                populated_llm_count += 1
                overall_precision += metrics["precision"]
            
            if human_value is not None:
                populated_human_count += 1
                overall_recall += metrics["recall"]
        
        except Exception as e:
            print(f"Error processing key {key}: {e}")
            continue

    # Create DataFrame
    metrics_df = pd.DataFrame(metrics_data)

    # Calculate average metrics
    avg_precision = overall_precision / populated_llm_count if populated_llm_count > 0 else 0
    avg_recall = overall_recall / populated_human_count if populated_human_count > 0 else 0

    # Print summary
    print(f"Average Precision (populated LLM attributes): {avg_precision:.2%}")
    print(f"Average Recall (populated human attributes): {avg_recall:.2%}")
    print(f"Total LLM populated attributes: {populated_llm_count}")
    print(f"Total human annotation populated attributes: {populated_human_count}")

    return metrics_df


## Main

In [51]:
# Sample data for demonstration
human_data = {
        "Key 1": "Value 1",
        "Key 2": [
            10,
            30
        ],
        "Key 3": [
            {
                "SubKey1": "Text A"
            },
            {
                "SubKey1": "Text Z"
            }
        ],
        "Key 4": "Another Val",
        "Key 5": [
            "Item1",
            "ItemX",
            "ItemY"
        ],
        "Key 6": [
            {
                "SubKey1": "This is a code to test"
            },
            {
                "SubKey1": "Text Y"
            }
        ]
}

llm_data = {
        "Key 1": "Value 1",
        "Key 2": [
            10,
            20
        ],
        "Key 3": [
            {
                "SubKey1": "This is a test"
            },
            {
                "SubKey1": "Text A"
            }
        ],
        "Key 5": [
            "Item1",
            "Item2"
        ],
        "Key 6": [
            {
                "SubKey1": "This is a testing code"
            },
            {
                "SubKey1": "Text D"
            }
        ],
        "Key 7":'Test'
}

In [52]:
print(json.dumps(human_data, indent=4))

{
    "Key 1": "Value 1",
    "Key 2": [
        10,
        30
    ],
    "Key 3": [
        {
            "SubKey1": "Text A"
        },
        {
            "SubKey1": "Text Z"
        }
    ],
    "Key 4": "Another Val",
    "Key 5": [
        "Item1",
        "ItemX",
        "ItemY"
    ],
    "Key 6": [
        {
            "SubKey1": "This is a code to test"
        },
        {
            "SubKey1": "Text Y"
        }
    ]
}


In [53]:
print(json.dumps(llm_data, indent=4))

{
    "Key 1": "Value 1",
    "Key 2": [
        10,
        20
    ],
    "Key 3": [
        {
            "SubKey1": "This is a test"
        },
        {
            "SubKey1": "Text A"
        }
    ],
    "Key 5": [
        "Item1",
        "Item2"
    ],
    "Key 6": [
        {
            "SubKey1": "This is a testing code"
        },
        {
            "SubKey1": "Text D"
        }
    ],
    "Key 7": "Test"
}


In [57]:
# Main execution
# Semantic similarity function using sentence-transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

try:
    metrics_df = compare_flat_dictionaries(human_data, llm_data)
    metrics_df
except Exception as e:
    print(f"Unexpected error in metrics calculation: {e}")

Average Precision (populated LLM attributes): 70.00%
Average Recall (populated human attributes): 55.56%
Total LLM populated attributes: 5
Total human annotation populated attributes: 6


In [58]:
metrics_df

,Key,Human Value,LLM Value,Match,Similarity,Precision,Recall
0,Key 1,Value 1,Value 1,✓,100.00%,1.0,1.000000
1,Key 2,"[10, 30]","[10, 20]",✓,100.00%,1.0,1.000000
2,Key 3,"[{'SubKey1': 'Text A'}, {'SubKey1': 'Text Z'}]","[{'SubKey1': 'This is a test'}, {'SubKey1': 'Text A'}]",✗,50.00%,0.5,0.500000
3,Key 4,Another Val,,✗,0%,0.0,0.000000
4,Key 5,"['Item1', 'ItemX', 'ItemY']","['Item1', 'Item2']",✗,41.67%,0.5,0.333333
5,Key 6,"[{'SubKey1': 'This is a code to test'}, {'SubKey1': 'Text Y'}]","[{'SubKey1': 'This is a testing code'}, {'SubKey1': 'Text D'}]",✗,50.00%,0.5,0.500000
